# Topic Analysis

In [20]:
import datetime

import pandas as pd
import spacy
import re
import string
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import nltk
from nltk.corpus import stopwords
from spacy.tokens import Token
from tqdm import tqdm


import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.coherencemodel import CoherenceModel

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from ipywidgets import interact

%matplotlib inline
np.random.seed(500)

In [2]:
df = pd.read_csv("..//data//Womens Clothing E-Commerce Reviews Sentiment v2.csv")

In [3]:
## Discarding a empty reviews records

df=df[((df.processed_Review_text.isna()==False) & (df.processed_Review_text.isnull()==False) & (df.processed_Review_text!=""))]

## Tokenizing the review text

In [4]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)
    return result

In [40]:
def buildLDA(processed_docs, ip_num_topics=4):
    dictionary = gensim.corpora.Dictionary(processed_docs)

    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]

    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=ip_num_topics, id2word=dictionary, passes=2, workers=4, alpha =1)
    
    return dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf

In [6]:
def LDAtopicSummary(model, displaysubplot=[2,2]):
    for idx, topic in model.print_topics(-1):
        print('Topic: {} Word: {}'.format(idx, topic))

In [7]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        #print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords',"Text"]
    return(sent_topics_df)


## LDA - Non-Recommendation 

### Non-Recommendation & Negative reviews

In [38]:
Neg_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]<0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_NR_processed_docs,8)
len(Neg_NR_processed_docs)

895

In [39]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Neg_NR_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.25511508246749526


In [9]:
#LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.009905 -0.049631       1        1  23.364635
4      0.016033  0.036923       2        1  19.033361
3     -0.025203  0.000785       3        1  14.133162
1      0.013880  0.001753       4        1  10.277392
0     -0.035949 -0.034950       5        1   9.295042
5      0.028907  0.023814       6        1   9.015212
6      0.061404 -0.017956       7        1   7.937769
7     -0.049166  0.039262       8        1   6.943432, topic_info=    Category        Freq          Term       Total  loglift  logprob
158  Default   50.000000        blouse   50.000000  30.0000  30.0000
45   Default   56.000000          poor   56.000000  29.0000  29.0000
57   Default   89.000000          wash   89.000000  28.0000  28.0000
11   Default  177.000000         small  177.000000  27.0000  27.0000
81   Default  165.000000          size  165.000000  26.0000  26.0000
47   Default  112.000000       quality  112.000000  25.0000  25.0000
9    Default   75.000000        pretty   75.000000  24.0000  24.0000
173  Default   42.000000       pattern   42.000000  23.0000  23.0000
5    Default   98.000000          feel   98.000000  22.0000  22.0000
30   Default  126.000000         color  126.000000  21.0000  21.0000
69   Default  103.000000       sweater  103.000000  20.0000  20.0000
134  Default   53.000000         heavy   53.000000  19.0000  19.0000
122  Default   36.000000        shrink   36.000000  18.0000  18.0000
89   Default   65.000000        length   65.000000  17.0000  17.0000
159  Default   31.000000          cold   31.000000  16.0000  16.0000
119  Default   79.000000        review   79.000000  15.0000  15.0000
29   Default   42.000000          blue   42.000000  14.0000  14.0000
112  Default   54.000000         weird   54.000000  13.0000  13.0000
162  Default   25.000000         water   25.000000  12.0000  12.0000
68   Default   71.000000      shoulder   71.000000  11.0000  11.0000
140  Default   33.000000          flat   33.000000  10.0000  10.0000
36   Default   61.000000        medium   61.000000   9.0000   9.0000
182  Default   62.000000         awful   62.000000   8.0000   8.0000
130  Default   40.000000          suit   40.000000   7.0000   7.0000
99   Default   91.000000          long   91.000000   6.0000   6.0000
4    Default  150.000000        fabric  150.000000   5.0000   5.0000
71   Default   56.000000          soft   56.000000   4.0000   4.0000
91   Default   58.000000          body   58.000000   3.0000   3.0000
152  Default   32.000000          jean   32.000000   2.0000   2.0000
31   Default   30.000000      delicate   30.000000   1.0000   1.0000
..       ...         ...           ...         ...      ...      ...
15    Topic8    6.354323        zipper   35.475166   0.9477  -4.8237
181   Topic8    7.996177          seam   46.452400   0.9079  -4.5939
150   Topic8    4.104984    absolutely   24.213009   0.8927  -5.2607
118   Topic8    6.892241       regular   41.296280   0.8770  -4.7425
57    Topic8   14.609133          wash   89.211998   0.8580  -3.9912
90    Topic8    3.853787          mean   24.613134   0.8132  -5.3238
132   Topic8    5.392456    completely   36.012157   0.7685  -4.9879
167   Topic8    5.579591         piece   37.766827   0.7551  -4.9537
175   Topic8    4.816441        casual   32.712654   0.7516  -5.1008
38    Topic8    5.569215          nice   38.124077   0.7438  -4.9556
183   Topic8    3.744265         point   27.446121   0.6754  -5.3526
203   Topic8    4.564402        jacket   33.763863   0.6663  -5.1546
69    Topic8   13.607180       sweater  103.153946   0.6417  -4.0623
179   Topic8    7.319822          sale   55.591141   0.6399  -4.6823
120   Topic8    7.166765          come   55.442223   0.6215  -4.7034
125   Topic8    5.163874         super   40.549011   0.6065  -5.0312
30    Topic8   15.246846         color  126.644737   0.5504  -3.9485
121   Topic8    5.790368         gre

### Non-Recommendation & Positive reviews

In [61]:
Pos_NR_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
Pos_NR_processed_docs_org=df["Review Text"][(df["Recommended IND"]==0 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_NR_processed_docs,9)

In [62]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Pos_NR_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.2617132427580249


In [76]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.000724 -0.000097       1        1  11.783358
3     -0.001584 -0.002435       2        1  11.381735
5      0.003246 -0.004115       3        1  11.207246
2     -0.003439 -0.001976       4        1  11.132832
6     -0.002671  0.002457       5        1  11.110199
0     -0.002342  0.002722       6        1  11.064251
1      0.003659  0.001797       7        1  11.013556
7     -0.000584 -0.001179       8        1  10.774499
8      0.002992  0.002825       9        1  10.532324, topic_info=    Category        Freq        Term       Total  loglift  logprob
3    Default  759.000000       dress  759.000000  30.0000  30.0000
139  Default  374.000000        cute  374.000000  29.0000  29.0000
35   Default  443.000000      return  443.000000  28.0000  28.0000
62   Default  335.000000     sweater  335.000000  27.0000  27.0000
69   Default  179.000000        come  179.000000  26.0000  26.0000
14   Default  244.000000      petite  244.000000  25.0000  25.0000
114  Default  492.000000      fabric  492.000000  24.0000  24.0000
154  Default  425.000000       shirt  425.000000  23.0000  23.0000
61   Default  246.000000      sleeve  246.000000  22.0000  22.0000
228  Default  296.000000       waist  296.000000  21.0000  21.0000
128  Default  265.000000     flatter  265.000000  20.0000  20.0000
18   Default  221.000000       tight  221.000000  19.0000  19.0000
16   Default  433.000000       small  433.000000  18.0000  18.0000
204  Default  246.000000    retailer  246.000000  17.0000  17.0000
15   Default  532.000000        size  532.000000  16.0000  16.0000
108  Default  261.000000      online  261.000000  15.0000  15.0000
21   Default  346.000000        work  346.000000  14.0000  14.0000
43   Default  349.000000        feel  349.000000  13.0000  13.0000
0    Default  212.000000       cheap  212.000000  12.0000  12.0000
25   Default  437.000000       color  437.000000  11.0000  11.0000
144  Default  180.000000      review  180.000000  10.0000  10.0000
283  Default  277.000000       model  277.000000   9.0000   9.0000
104  Default  176.000000        time  176.000000   8.0000   8.0000
161  Default  216.000000       photo  216.000000   7.0000   7.0000
257  Default  167.000000       white  167.000000   6.0000   6.0000
70   Default  150.000000  definitely  150.000000   5.0000   5.0000
87   Default  361.000000       great  361.000000   4.0000   4.0000
36   Default  251.000000       skirt  251.000000   3.0000   3.0000
117  Default  228.000000    shoulder  228.000000   2.0000   2.0000
47   Default  539.000000        like  539.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
204   Topic9   32.019703    retailer  246.330963   0.2104  -5.1985
25    Topic9   54.292831       color  437.609497   0.1638  -4.6704
16    Topic9   52.988045       small  433.617065   0.1486  -4.6948
21    Topic9   41.698944        work  346.764496   0.1326  -4.9344
96    Topic9   38.007679        good  321.851105   0.1144  -5.0271
29    Topic9   14.750708        idea  110.290329   0.2389  -5.9735
38    Topic9   20.039238        tall  156.704193   0.1941  -5.6671
63    Topic9   41.176399       think  359.614685   0.0836  -4.9470
40    Topic9   49.878777        wear  451.616302   0.0475  -4.7552
20    Topic9   40.017853        want  353.975372   0.0708  -4.9755
87    Topic9   40.634151       great  361.935425   0.0639  -4.9602
306   Topic9   25.006849        bust  207.765732   0.1335  -5.4457
112   Topic9   37.586067   beautiful  343.763489   0.0374  -5.0382
14    Topic9   28.365370      petite  244.448532   0.0969  -5.3197
43    Topic9   37.673157        feel  349.366058   0.0235  -5.0359
60    Topic9   39.250278       short  370.518372   0.0058  -4.9949
62    Topic9   36.059982     sweater  335.536987   0.0202  -5.0797
37    Topic9   28.029249       style  245.364273   0.0812  -5.3316
12    Topic9 

In [75]:
def demo(title):
    if not np.isnan(title):
        return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title-1].tolist()][:5])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_NR_processed_docs_org) )
interact(demo, title=np.sort(df_dominant_topic["Dominant_Topic"].unique())+1);

interactive(children=(Dropdown(description='title', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0), val…

## LDA - Recommended Data

### Recommendation & Negative reviews

In [77]:
Neg_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)
Neg_R_processed_docs_org=df["Review Text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]<0)].map(preprocess)

dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Neg_R_processed_docs,10)
len(Neg_R_processed_docs)

820

In [80]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=lda_model_tfidf, texts=Neg_R_processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3023114596715811


In [78]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.001188  0.003511       1        1  10.479258
6      0.001126 -0.004624       2        1  10.306499
7     -0.002096 -0.002293       3        1  10.161350
1     -0.002406  0.004277       4        1  10.120184
5     -0.003153 -0.000452       5        1  10.117647
4     -0.002997 -0.002313       6        1   9.886766
3     -0.001683  0.002402       7        1   9.856924
8      0.004693  0.000655       8        1   9.728037
0     -0.000071 -0.000988       9        1   9.719987
2      0.005399 -0.000174      10        1   9.623342, topic_info=    Category        Freq        Term       Total  loglift  logprob
4    Default  130.000000        love  130.000000  30.0000  30.0000
11   Default  108.000000     usually  108.000000  29.0000  29.0000
61   Default  109.000000       tight  109.000000  28.0000  28.0000
73   Default   97.000000        feel   97.000000  27.0000  27.0000
26   Default   42.000000  compliment   42.000000  26.0000  26.0000
95   Default   73.000000      pretty   73.000000  25.0000  25.0000
24   Default  209.000000       small  209.000000  24.0000  24.0000
7    Default  191.000000        size  191.000000  23.0000  23.0000
14   Default  121.000000      fabric  121.000000  22.0000  22.0000
63   Default  154.000000        look  154.000000  21.0000  21.0000
9    Default   83.000000       style   83.000000  20.0000  20.0000
62   Default   66.000000       green   66.000000  19.0000  19.0000
5    Default  136.000000       order  136.000000  18.0000  18.0000
134  Default   76.000000       white   76.000000  17.0000  17.0000
19   Default  166.000000        like  166.000000  16.0000  16.0000
28   Default  114.000000       shirt  114.000000  15.0000  15.0000
29   Default  105.000000       black  105.000000  14.0000  14.0000
54   Default  113.000000        long  113.000000  13.0000  13.0000
16   Default   96.000000       large   96.000000  12.0000  12.0000
114  Default   34.000000        year   34.000000  11.0000  11.0000
10   Default   73.000000       usual   73.000000  10.0000  10.0000
109  Default   88.000000        jean   88.000000   9.0000   9.0000
50   Default  131.000000       color  131.000000   8.0000   8.0000
1    Default  223.000000       dress  223.000000   7.0000   7.0000
15   Default   39.000000        knee   39.000000   6.0000   6.0000
191  Default   59.000000        blue   59.000000   5.0000   5.0000
125  Default   88.000000     sweater   88.000000   4.0000   4.0000
105  Default   84.000000        want   84.000000   3.0000   3.0000
174  Default   33.000000       wrong   33.000000   2.0000   2.0000
153  Default   26.000000       pound   26.000000   1.0000   1.0000
..       ...         ...         ...         ...      ...      ...
150  Topic10    3.673712        good   30.501955   0.2244  -5.7205
171  Topic10    3.920164        hate   32.667465   0.2207  -5.6556
99   Topic10    6.091401       print   50.793949   0.2201  -5.2149
137  Topic10    5.048810        cozy   42.251213   0.2165  -5.4026
35   Topic10    6.768377       price   56.646248   0.2164  -5.1095
63   Topic10   17.851585        look  154.866150   0.1805  -4.1396
125  Topic10   10.308528     sweater   88.737823   0.1883  -4.6888
1    Topic10   24.465044       dress  223.635666   0.1282  -3.8245
110  Topic10   11.184101      medium  103.091370   0.1199  -4.6072
7    Topic10   18.776983        size  191.944336   0.0164  -4.0891
4    Topic10   13.409827        love  130.424973   0.0662  -4.4258
105  Topic10    9.216600        want   84.605560   0.1240  -4.8007
96   Topic10   10.452411       short  100.692772   0.0757  -4.6749
3    Topic10   15.084391      little  162.773727  -0.0377  -4.3081
19   Topic10   15.113333        like  166.337646  -0.0575  -4.3062
28   Topic10   11.343771       shirt  114.230217   0.0314  -4.5931
61   Topic10   10.882736       tight  109.429764   0.0329  -4.6346
29   Topic10   10.465854  

In [79]:
def demo(title):
    if not np.isnan(title):
        return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title-1].tolist()][:5])

df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Neg_R_processed_docs_org) )
interact(demo, title=np.sort(df_dominant_topic["Dominant_Topic"].unique())+1);

interactive(children=(Dropdown(description='title', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0…

### Recommendation & Positive reviews

In [16]:
Pos_R_processed_docs=df["processed_Review_text"][(df["Recommended IND"]==1 ) & (df["PA_Polarity"]>0)].map(preprocess)
dictionary,bow_corpus,corpus_tfidf, lda_model_tfidf =buildLDA(Pos_R_processed_docs,12)
len(Pos_R_processed_docs)

17605

In [17]:
pyLDAvis.gensim.prepare(lda_model_tfidf, bow_corpus, dictionary=lda_model_tfidf.id2word)

C:\Users\Dell\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.041582 -0.015740       1        1  14.558821
5     -0.028467 -0.004393       2        1  10.234323
1      0.017714  0.016635       3        1   9.970642
7      0.029830  0.020075       4        1   9.750823
3      0.023300 -0.000903       5        1   8.078131
10    -0.010156 -0.000140       6        1   7.661788
6      0.004483  0.020346       7        1   7.632018
2     -0.007487  0.009916       8        1   7.323108
9      0.030186 -0.054562       9        1   7.196839
8     -0.003733  0.016037      10        1   6.324891
0     -0.005579 -0.008685      11        1   5.890244
11    -0.008508  0.001413      12        1   5.378365, topic_info=     Category         Freq         Term        Total  loglift  logprob
642   Default   271.000000       romper   271.000000  30.0000  30.0000
1032  Default   356.000000         suit   356.000000  29.0000  29.0000
22    Default  1125.000000   compliment  1125.000000  28.0000  28.0000
35    Default  1887.000000        shirt  1887.000000  27.0000  27.0000
32    Default   910.000000      legging   910.000000  26.0000  26.0000
201   Default   944.000000        comfy   944.000000  25.0000  25.0000
211   Default  1753.000000      sweater  1753.000000  24.0000  24.0000
49    Default  1472.000000        skirt  1472.000000  23.0000  23.0000
6     Default  3865.000000        dress  3865.000000  22.0000  22.0000
31    Default  1473.000000   flattering  1473.000000  21.0000  21.0000
178   Default  1898.000000         cute  1898.000000  20.0000  20.0000
84    Default   374.000000      fitting   374.000000  19.0000  19.0000
1360  Default   148.000000       kimono   148.000000  18.0000  18.0000
45    Default  1505.000000         pant  1505.000000  17.0000  17.0000
26    Default   312.000000     jumpsuit   312.000000  16.0000  16.0000
170   Default   971.000000        price   971.000000  15.0000  15.0000
328   Default   366.000000    extremely   366.000000  14.0000  14.0000
212   Default   767.000000         warm   767.000000  13.0000  13.0000
123   Default   417.000000        ankle   417.000000  12.0000  12.0000
1     Default  2158.000000  comfortable  2158.000000  11.0000  11.0000
164   Default   375.000000       simple   375.000000  10.0000  10.0000
56    Default   949.000000        loose   949.000000   9.0000   9.0000
39    Default  2547.000000        color  2547.000000   8.0000   8.0000
147   Default   444.000000         coat   444.000000   7.0000   7.0000
295   Default  1244.000000       summer  1244.000000   6.0000   6.0000
769   Default   571.000000        worth   571.000000   5.0000   5.0000
519   Default   288.000000       closet   288.000000   4.0000   4.0000
430   Default   472.000000      stylish   472.000000   3.0000   3.0000
125   Default  1453.000000         good  1453.000000   2.0000   2.0000
20    Default  1373.000000         true  1373.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
6     Topic12   176.843033        dress  3865.381836  -0.1618  -4.6485
20    Topic12   103.030128         true  1373.894897   0.3324  -5.1888
15    Topic12   129.292358        order  2209.200684   0.0845  -4.9617
43    Topic12   138.505508         look  2578.611816  -0.0013  -4.8929
56    Topic12    84.748558        loose   949.499756   0.5065  -5.3841
443   Topic12    59.295532         room   466.543579   0.8600  -5.7413
35    Topic12   114.182602        shirt  1887.032227   0.1178  -5.0860
42    Topic12   126.745697         like  2367.311768  -0.0045  -4.9816
278   Topic12    65.099007        sheer   579.175293   0.7371  -5.6479
28    Topic12   134.854095         wear  2801.034180  -0.1108  -4.9196
12    Topic12   138.230789         love  3103.986084  -0.1887  -4.8949
231   Topic12   112.519112         soft  2006.631104   0.0417  -5.1007
108   Topic12    73.529396         body   789.026001   0.5497  -5.5261
11    

In [18]:
# LDAtopicSummary(lda_model_tfidf, [2,4])
df_dominant_topic = format_topics_sentences(lda_model_tfidf,bow_corpus,list(Pos_R_processed_docs) )
def demo(title):
    return([" ".join(i) for i in df_dominant_topic["Text"][df_dominant_topic["Dominant_Topic"]==title].tolist()][:20])

interact(demo, title=range(12));

interactive(children=(Dropdown(description='title', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11), value=0), …